In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee

ee.Authenticate()
ee.Initialize()

This tool has been deprecated, use 'gio open' instead.
See 'gio help open' for more info.



Enter verification code: 4/1ARtbsJrsqjWrFWtD1lwVOt3hR0b7rjICZHD6aTxAU_j_fu1eNuhjPXs1PjE

Successfully saved authorization token.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp

%matplotlib inline

In [4]:
geoJSON = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [
                    [-46.65810255733541, -23.535732559501554],
                    [-46.603706283614684, -23.535732559501554],
                    [-46.603706283614684, -23.575921045095285],
                    [-46.65810255733541, -23.575921045095285],
                ],
                "type": "LineString",
            },
        }
    ],
}

In [140]:
import pandas as pd

In [141]:
coords_df = pd.read_csv("/home/tim/data/height_model_file_edges.csv")

In [142]:
coords_df.head()

,file_name,left,right,top,bottom,pixel_horiz,pixel_vert
0,2222-132,318342.09,318468.09,7372316.75,7372138.25,253,358
1,2222-211,318457.24,318999.24,7372323.51,7371857.51,1085,933
2,2222-212,318988.17,319403.67,7372330.12,7371868.12,832,925
3,2222-222,320292.39,320591.89,7372319.53,7371756.03,600,1128
4,2222-224,320203.08,320495.58,7371766.54,7371370.54,586,793


In [172]:
coords_1 = coords_df.loc[[500]]

In [173]:
coords = [
    [coords_1.left.values[0], coords_1.bottom.values[0]],
    [coords_1.left.values[0], coords_1.top.values[0]],
    [coords_1.right.values[0], coords_1.top.values[0]],
    [coords_1.right.values[0], coords_1.bottom.values[0]],
]

In [33]:
# import rasterio.warp

In [38]:
# x = np.asarray([coords[0][0]])
# y = np.asarray([coords[0][1]])

In [41]:
# destination_crs = "EPSG:3857"
# src_crs = "EPSG:31983"

In [42]:
# geometry = rasterio.warp.transform(
#    src_crs,
#    destination_crs,
#    x,
#    y
# )

In [44]:
maxar_df = pd.read_csv("/home/tim/data/UNICEF_data/grids_extent.csv")
maxar_df.head()

,file_id,qmdt_cod,top,left,bottom,right
0,0001.png,3212-353,-2.727409e+06,-5.196069e+06,-2.728042e+06,-5.195489e+06
1,0002.png,3212-344,-2.727409e+06,-5.196649e+06,-2.728042e+06,-5.196069e+06
2,0003.png,3212-343,-2.727409e+06,-5.197229e+06,-2.728042e+06,-5.196649e+06
3,0004.png,3211-464,-2.727409e+06,-5.197809e+06,-2.728042e+06,-5.197229e+06
4,0005.png,3211-463,-2.727409e+06,-5.198388e+06,-2.728042e+06,-5.197809e+06


In [71]:
c = 1000

In [112]:
coords = [
    [maxar_df.left.values[c], maxar_df.bottom.values[c]],
    [maxar_df.left.values[c], maxar_df.top.values[c]],
    [maxar_df.right.values[c], maxar_df.top.values[c]],
    [maxar_df.right.values[c], maxar_df.bottom.values[c]],
]

In [174]:
coords = np.array(coords)

In [175]:
print(np.shape(coords))

(4, 2)


In [176]:
print(coords)

[[ 317861.15 7376920.75]
 [ 317861.15 7377507.75]
 [ 318310.65 7377507.75]
 [ 318310.65 7376920.75]]


In [177]:
from pyproj import Transformer

In [178]:
transformer = Transformer.from_crs("EPSG:31983", "EPSG:4326", always_xy=True)

In [179]:
coords_x, coords_y = transformer.transform(coords[:, 0], coords[:, 1])

In [180]:
coords_new = np.dstack([coords_x, coords_y])[0]

In [181]:
print(coords_new)

[[-46.78654659 -23.70834814]
 [-46.78647442 -23.70304857]
 [-46.78206724 -23.70309939]
 [-46.78213924 -23.70839897]]


In [182]:
coords_new = coords_new.tolist()

In [183]:
aoi = ee.Geometry.Polygon(coords_new)

In [184]:
ffa_db = ee.Image(
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(aoi)
    .filterDate(ee.Date("2020-08-01"), ee.Date("2020-08-31"))
    .first()
    .clip(aoi)
)

In [186]:
url = ffa_db.select(["B2", "B3", "B4"]).getThumbURL({"min": 0, "max": 3000})
disp.Image(url=url, width=200)